# Couette (cilindros coaxiais)

Queremos calcular a viscosidade dinâmica (μ) e cinemática (ν) num ensaio tipo Couette com um cilindro exterior em rotação e o interior mantido imóvel por uma força que é medida através de um dinamómetro.

Queremos: 
$\mu$ - viscosidade dinâmica
e
$\nu$ - viscosidade cinemática

A experiência de Couette é uma abstração que permite representar dois planos infinitos teóricos usando dois cilindros concentricos ignorando efeitos de fronteira e de tal forma que todas as forças envolvidas são na direcção principal do movimento.

Sabemos que:

$
\mu=\tau\frac{dy}{dv}
$ 

sendo $\tau$ a tensão tangencial, $dy$ a dimensão da folga infinitesimal entre planos paralelos, e $dv$ a variação de velocidade entre os dois planos. Sabendo que a tensão tangencial $\tau$ é 

$
\tau=\frac{F}{A}
$

, com $F$ a força exercida sobre o plano estático e $A$ a superficie de contacto, é possivel deduzir que:

$
\mu=\frac{F}{A}\frac{dy}{dv}
$

Para calcular a viscosidade cinemática dividimos a viscosidade dinâmica $\mu$ pela massa volumica do fluido $\rho$

$
\nu=\frac{\mu}{\rho}
$

## Dados fornecidos: 

$\gamma=880 \:[kg\:m^{-3}]$;

$r=0.15\:[m]$;

$h=0.4\:[m]$;

$dy=0.1\times10^{-3} \:[m]$;

$v_{ext}=70\:[rot\:min^{-1}]$;

$v_{int}=0 \:[m\:s^{-1}]$;

$F=27.6 \:[N]$


# Passos de cálculo para os dados fornecidos

## Transformação da velocidade angular em linear e cálculo do dv

$v_{ext}=\omega \times r$

$\omega=\frac{70[rot\:min^{-1}]\times 2 \pi [rad\:rot^{-1}]}{60 [s\: min^{-1}]}=7.33 [rad\:s^{-1}]$

$v_{ext}= {omega:.6g} 7.33\:[rad\:s^{-1}] \times 0.15\:[m]=1.0995 \:[m\:s^{-1}]$ 

Nota: $rad$ é uma unidade adimensional ou $[m/m]$

$dv=v_{ext}-v_{int} = 1.0995 \:[m\:s^{-1}]$

## Cálculo da viscosidade dinâmica

Assumindo $\mu=\tau\frac{dy}{dv}$ e $\tau=\frac{F}{A}$

Obtém-se que $\mu=\frac{F}{A}\frac{dy}{dv}$, substituindo

$\mu=\frac{27.6 \:[N]}{2*\pi*0.15\:[m]*0.4\:[m]}\frac{0.1\times10^{-3} \:[m]}{1.0995 \:[m\:s^{-1}]}=6.658\times 10^{-3}\:[N\:s\:m^{-2}]$

## Cálculo da viscosidade cinemática

$\nu=\frac{6.658\:[N\:s\:m^{-2}]}{880 \:[kg\:m^{-3}]}=7.566\:[m^{2}\:s^{-1}]$


In [1]:
%matplotlib inline

import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Output
from IPython.display import display, Markdown, clear_output
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots

# ---------- Funções ----------
def rpm_to_rad_s(rpm):
    return rpm * 2*math.pi / 60.0

def mu_small_gap(F, e, r, h, omega_rad_s):
    return (F*e) / (2*math.pi*omega_rad_s*(r**2)*h)

def mu_exact_cylinders(F, e, r, h, omega_rad_s):
    R1 = r
    R2 = r + e
    return (F*(R2**2 - R1**2)) / (4*math.pi*h*omega_rad_s*(R2**2)*R1)

def utheta_profile(R, R1, R2, omega, perfil_exato=True):
    # Condições: u(R1)=0, u(R2)=omega*R2
    if not perfil_exato:
        return (omega*R2 / max(R2-R1, 1e-15)) * (R - R1)
    # Exato (Couette cilíndrico): u(R) = A R + B/R
    A = omega*(R2**2) / max((R2**2 - R1**2), 1e-30)
    B = -A*(R1**2)
    return A*R + B/np.maximum(R, 1e-30)

# ---------- Sliders ----------
F_w = widgets.FloatSlider(value=27.6, min=0.0, max=200.0, step=0.1, description='F (N)', continuous_update=False)
rho_w = widgets.FloatSlider(value=880.0, min=200.0, max=2000.0, step=10.0, description='ρ (kg/m³)', continuous_update=False)
r_w = widgets.FloatSlider(value=0.15, min=0.01, max=1.0, step=0.005, description='r (m)', continuous_update=False)
h_w = widgets.FloatSlider(value=0.40, min=0.01, max=2.0, step=0.01, description='h (m)', continuous_update=False)
e_w = widgets.FloatLogSlider(value=1e-4, base=10, min=-6, max=-2, step=0.05, description='e (m)', continuous_update=False)
omega_w = widgets.FloatSlider(value=70.0, min=0.0, max=500.0, step=1.0, description='ω (rpm)', continuous_update=False)

usar_mu_exata_w = widgets.Checkbox(value=False, description='μ: solução exata')
perfil_exato_w = widgets.Checkbox(value=True, description='Perfil uθ: exato')

# Qual perfil no gráfico 2D?
eixo_y_w = widgets.Dropdown(options=[("uθ vs R", "R"), ("uθ vs y=R-r", "y")], value="y", description="2D:")

ui = VBox([
    HBox([F_w, rho_w]),
    HBox([r_w, h_w]),
    HBox([e_w, omega_w]),
    HBox([usar_mu_exata_w, perfil_exato_w, eixo_y_w]),
])

out_plot = Output()
out_txt = Output()
display(VBox([ui, out_plot, out_txt]))

def draw():
    F = float(F_w.value)
    rho = float(rho_w.value)
    r = float(r_w.value)
    h = float(h_w.value)
    e = float(e_w.value)
    omega_rpm = float(omega_w.value)
    omega = rpm_to_rad_s(omega_rpm)

    R1 = r
    R2 = r + e

    # viscosidades/tensão (a partir do F medido)
    mu = mu_exact_cylinders(F, e, r, h, omega) if usar_mu_exata_w.value else mu_small_gap(F, e, r, h, omega)
    nu = mu / rho if rho > 0 else float("nan")
    A = 2*math.pi*r*h
    tau = F / A if A > 0 else float("nan")

    # --- dados para 3D (cilindros + 1 corte radial para ser leve) ---
    ntheta, nz = 80, 40
    th = np.linspace(0, 2*np.pi, ntheta)
    zz = np.linspace(0, h, nz)
    TH, ZZ = np.meshgrid(th, zz)

    X1 = R1*np.cos(TH); Y1 = R1*np.sin(TH); Z1 = ZZ
    X2 = R2*np.cos(TH); Y2 = R2*np.sin(TH); Z2 = ZZ

    # corte radial em theta=0
    nR = 140
    Rline = np.linspace(R1, R2, nR)
    RR, ZZs = np.meshgrid(Rline, zz)
    Xs = RR
    Ys = np.zeros_like(RR)
    Zs = ZZs
    U = utheta_profile(RR, R1, R2, omega, perfil_exato=perfil_exato_w.value)

    umax = float(np.nanmax(U)) if np.isfinite(np.nanmax(U)) else 1.0
    norm = colors.Normalize(vmin=0.0, vmax=max(umax, 1e-12))
    facecolors = cm.viridis(norm(U))

    # --- dados para 2D ---
    Uline = utheta_profile(Rline, R1, R2, omega, perfil_exato=perfil_exato_w.value)
    x2d = Rline if eixo_y_w.value == "R" else (Rline - R1)
    xlab = "R (m)" if eixo_y_w.value == "R" else "y = R − r (m)"

    with out_plot:
        clear_output(wait=True)
        fig = plt.figure(figsize=(12, 5))
        ax3d = fig.add_subplot(1, 2, 1, projection="3d")
        ax2d = fig.add_subplot(1, 2, 2)

        # 3D: cilindros
        ax3d.plot_surface(X1, Y1, Z1, color="gray", alpha=0.12, linewidth=0)
        ax3d.plot_surface(X2, Y2, Z2, color="gray", alpha=0.12, linewidth=0)

        # 3D: corte radial com cores
        ax3d.plot_surface(Xs, Ys, Zs, facecolors=facecolors, shade=False, linewidth=0, antialiased=False)

        sm = cm.ScalarMappable(norm=norm, cmap=cm.viridis)
        sm.set_array([])
        fig.colorbar(sm, ax=ax3d, shrink=0.7, pad=0.08, label="uθ (m/s)")

        ax3d.set_title("3D: cilindros + corte radial")
        ax3d.set_xlabel("x (m)")
        ax3d.set_ylabel("y (m)")
        ax3d.set_zlabel("z (m)")
        lim = (R2 if R2 > 0 else 1e-6) * 1.25
        ax3d.set_xlim(-lim, lim)
        ax3d.set_ylim(-lim, lim)
        ax3d.set_zlim(0, max(h, 1e-6))
        ax3d.view_init(elev=18, azim=35)

        # 2D: perfil
        ax2d.plot(x2d, Uline, lw=2)
        ax2d.set_title("Perfil de velocidades")
        ax2d.set_xlabel(xlab)
        ax2d.set_ylabel("uθ (m/s)")
        ax2d.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()
        show_inline_matplotlib_plots()

    with out_txt:
        clear_output(wait=True)
        display(Markdown(
            f"- ω = {omega_rpm:.4g} rpm = {omega:.6g} rad/s  \n"
            f"- R1 = r = {R1:.6g} m, R2 = r+e = {R2:.6g} m  \n"
            f"- U(parede externa) = ω R2 = {(omega*R2):.6g} m/s  \n"
            f"- A = 2π r h = {A:.6g} m²  \n"
            f"- τ = F/A = {tau:.6g} Pa  \n"
            f"- μ = {mu:.6g} Pa·s  \n"
            f"- ν = {nu:.6g} m²/s"
        ))

def on_change(change=None):
    draw()

for w in [F_w, rho_w, r_w, h_w, e_w, omega_w, usar_mu_exata_w, perfil_exato_w, eixo_y_w]:
    w.observe(on_change, names="value")

draw()